In [1]:
from graphviz import Digraph
import os


# --- Create a flowchart ---
dot = Digraph(comment="Pipeline Flowchart")

dot.attr(rankdir='TB', size='8,5')

# Nodes
dot.node('A', 'Upload Data')
dot.node('B', 'Preprocessing')
dot.node('C', 'Train Model')
dot.node('D', 'Evaluate Model')
dot.node('E', 'Visualize Results\n(GradCAM & SHAP)')
dot.node('F', 'Download Reports')

# Edges
dot.edges(['AB', 'BC', 'CD', 'DE', 'EF'])

# Save the flowchart
dot.render('demo_assets/pipeline_flowchart', format='png', cleanup=True)

print("✅ Flowchart saved at /demo_assets/pipeline_flowchart.png")


✅ Flowchart saved at /demo_assets/pipeline_flowchart.png


In [4]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import shutil
import sys
import sys
sys.path.append("../scripts")  
from gradcam_utils import GradCAM


# Ensure access to scripts/
sys.path.append("../scripts")
from gradcam_utils import GradCAM

# --- Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("demo_assets", exist_ok=True)

# --- 1. Copy a Sample Raw Image (WSI simulation) ---
raw_src = r"C:\Users\frank\lymphoid_malignancy_project\data\raw\CLL\sj-03-476_001.tif"
raw_dst = "demo_assets/sample_raw1.png"
with Image.open(raw_src) as img:
    img.thumbnail((800, 800))
    img.save(raw_dst)
print("✅ Saved demo_assets/sample_raw1.png")

# --- 2. Copy a Sample Tile Image ---
tile_src = r"C:\Users\frank\lymphoid_malignancy_project\data\tiles\CLL\tile_0_x0_y0.png"
tile_dst = "demo_assets/sample_tile1.png"
shutil.copy(tile_src, tile_dst)
print("✅ Saved demo_assets/sample_tile1.png")

# --- 3. Copy + Resize Preprocessed Tile Image ---
processed_src = r"C:\Users\frank\lymphoid_malignancy_project\data\processed_tiles\CLL\tile_0_x0_y0_aug0.png"
processed_dst = "demo_assets/sample_processed_tile1.png"
with Image.open(processed_src) as img:
    img_resized = img.resize((224, 224))
    img_resized.save(processed_dst)
print("✅ Saved demo_assets/sample_processed_tile1.png")

# --- 4. GradCAM ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])
input_tensor = transform(img_resized).unsqueeze(0).to(device)

# --- Load Sample Image (Preprocessed Tile) ---
tile_path = r"C:\Users\frank\lymphoid_malignancy_project\data\processed_tiles\CLL\tile_0_x0_y0_aug0.png"
img = Image.open(tile_path).convert("RGB")
img_resized = img.resize((224, 224))  # <-- This is now defined
img_resized.save("demo_assets/sample_processed_tile1.png")

# --- Preprocess ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])
input_tensor = transform(img_resized).unsqueeze(0).to(device)

# --- Load ResNet50 Model ---
model_path = r"C:\Users\frank\lymphoid_malignancy_project\data\models\trained_resnet50.pth"
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 3)  # Adjust to match your class count
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

# --- Generate GradCAM ---
try:
    target_layer = model.layer4[1].conv2  # Works for ResNet50 too
    gradcam = GradCAM(model, target_layer)
    heatmap = gradcam.generate(input_tensor)

    img_np = np.array(img_resized)
    heatmap_resized = cv2.resize(heatmap, (img_np.shape[1], img_np.shape[0]))
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
    superimposed_img = heatmap_color * 0.4 + img_np

    gradcam_out = Image.fromarray(superimposed_img.astype(np.uint8))
    gradcam_out.save("demo_assets/sample_gradcam_output.png")
    print("✅ GradCAM saved to demo_assets/sample_gradcam_output.png")

except Exception as e:
    print("❌ GradCAM failed:", e)

# --- Copy SHAP Summary Plot ---
shap_source = r"C:\Users\frank\lymphoid_malignancy_project\outputs\shap_summary.png"
shap_dest = "demo_assets/sample_shap_summary.png"

try:
    shutil.copy(shap_source, shap_dest)
    print("✅ SHAP summary plot copied to demo_assets/sample_shap_summary.png")
except Exception as e:
    print("❌ Failed to copy SHAP plot:", e)

✅ Saved demo_assets/sample_raw1.png
✅ Saved demo_assets/sample_tile1.png
✅ Saved demo_assets/sample_processed_tile1.png


c:\Users\frank\lymphoid_malignancy_project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\frank\lymphoid_malignancy_project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\frank\lymphoid_malignancy_project\.venv\Lib\site-packages\torch\nn\modules\module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains m

✅ GradCAM saved to demo_assets/sample_gradcam_output.png
❌ Failed to copy SHAP plot: [Errno 2] No such file or directory: 'C:\\Users\\frank\\lymphoid_malignancy_project\\outputs\\shap_summary.png'


C:\Users\frank\AppData\Local\Temp\ipykernel_13752\651214457.py:84: RuntimeWarning: invalid value encountered in cast
  heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
